# BERTopic

In [1]:
import sys
sys.path.insert(0, '..')
from utils.preprocesamiento import df_to_list

In [2]:
from utils.cargar import df_caso
from utils.preprocesamiento import process_df, procesar_adela

caso = 'adela'
df = df_caso(caso)

df = procesar_adela(df)
df = df[df['opt_left'] == 'Producir el alimento contra déficit vitamínico']

df = process_df(df,'comment','sel',verbose=True)

26 rows found with non string elements for column comment (0.40%)
Deleting 685 columns for which max target value is over 7 (10.58%)
5761 available rows after processing


In [3]:
docs = df_to_list(df,'comment')

## BERTopic algorithm

In [2]:
from IPython.display import Image, display
display(Image(url='https://maartengr.github.io/BERTopic/algorithm/modularity.svg'))

In [ ]:
from bertopic import BERTopic

In [4]:
try:
    model = BERTopic.load("BT-{}.bertopic".format(caso))
except ValueError:
    model = BERTopic(language="spanish")
    topics, probs = model.fit_transform(docs)

/home/camilo/miniconda3/envs/ethics_env/lib/python3.9/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/camilo/miniconda3/envs/ethics_env/lib/python3.9/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/camilo/miniconda3/envs/ethics_env/lib/python3.9/site-pac

In [5]:
model.get_topic_freq().head(5)

,Topic,Count
0,-1,1084
4,0,1057
7,1,988
10,2,518
5,3,467


In [6]:
model.get_topic(0)[:10]

[('fruta', 0.027188619245504763),
 ('fruto', 0.02323425629387061),
 ('la', 0.021278591478337137),
 ('de', 0.021272313849935512),
 ('que', 0.02105168418425007),
 ('no', 0.018975306487465735),
 ('el', 0.0186605977703611),
 ('se', 0.01860758082390726),
 ('un', 0.015932227389658642),
 ('en', 0.015844316317521874)]

Topicos para algunos documentos

In [7]:
for i, t in enumerate(model.topics_[:10]):
    print('Tópico {}:\ttexto: {}'.format(t,docs[i]))

Tópico -1:	texto: Adela debería priorizar la salud de las personas antes que alguna creencia de un grupo pequeño de personas.
Tópico 4:	texto: Resguardar la identidad del pueblo originario significa un ahorro del recurso a largo plazo y en tanto que para conseguir a vitamina D y así suplir el deficit existen diversas fuentes, no es de extrema urgencia la producción de este fruto.
Tópico 25:	texto: creo que las creencias religiosas o tradiciones no deberian interferir en los avances cientificos
Tópico 22:	texto: El objetivo del proyecto no es mutuamente excluyente con las tradiciones locales, es decir, puede encontrarse una manera en que se pueda a la vez obtener el recurso y que la población del sector pueda seguir practicando su cultura.
Tópico 0:	texto: Se deben preservar las tradiciones de los pueblos originarios, es un tema de respeto hacia la gente que ha vivido en la zona, ya que a ellos les pertenece la fruta por generaciones. Si ellos dijeron que no estaban de acuerdo se debe r

Visualización

In [8]:
model.visualize_topics()

Podemos relacionar tópicos con palabras en específico

In [19]:
target = 'integridad'

topic_num = 5

similar_topics, similarity = model.find_topics(target, top_n=topic_num)
similar_topics

[21, 26, 14, 17, 15]

In [10]:
similarity

[0.49942476, 0.309725, 0.30662423, 0.2900588, 0.28991252]

In [22]:
data = [[] for _ in range(topic_num)]
max_w = 5

for i, t in enumerate(similar_topics):
    data[i].append(similarity[i])
    words_top = model.get_topics(t)
    data[i] += [tup[0] for tup in model.get_topic(t)[:max_w]]

In [23]:
from utils.exploracion import print_table_md

print("Tópicos relevantes para concepto: {}".format(target))

headers = ['Probabilidad'] + ['Palabra {}'.format(i+1) for i in range(max_w)]

print_table_md(headers, data)

Tópicos relevantes para concepto: integridad


| Probabilidad | Palabra 1 | Palabra 2 | Palabra 3 | Palabra 4 | Palabra 5 |
| --- | --- | --- | --- | --- | --- |
| 0.49942476 | misma | mitad | mismo | rellenare | idem |
| 0.309725 | respetar | pueblos | originarios | cuenta | importante |
| 0.30662423 | identitarias | tradiciones | las | resguardar | que |
| 0.2900588 | proyecto | no | que | es | importante |
| 0.28991252 | mi | opinion | postura | mantiene | opinión |


Guardando el modelo

In [25]:
model.save("BT-{}.bertopic".format(caso))

Para cargar:

`my_model = BERTopic.load("BT-caso")`